In [ ]:
# 它可以被用来预处理用户输入的任何方式。

In [1]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    # model_name="qwen2-1.5b-instruct",
    model_name="qwen2-plus",
    temperature=0,
    streaming=True,
)

ms_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('MOONSHOT_API_BASE'),
    openai_api_key=os.getenv('MOONSHOT_API_KEY'),
    model_name="moonshot-v1-8k",
    temperature=0,
    streaming=True,
)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    model_name="@cf/meta/llama-3-8b-instruct",
    temperature=0,
    streaming=True,
)

groq_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('GROQ_API_BASE'),
    openai_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama3-8b-8192",
    temperature=0,
    streaming=True,
)

In [2]:
from langchain_community.embeddings import DashScopeEmbeddings
from dotenv import load_dotenv
import os

load_dotenv(override=True)
qw_embedding = DashScopeEmbeddings(
    model="text-embedding-v2", dashscope_api_key=os.getenv('DASHSCOPE_API_KEY')
)

In [4]:
import logging

from langchain.retrievers import RePhraseQueryRetriever
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.re_phraser").setLevel(logging.INFO)

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=qw_embedding)

In [ ]:
DEFAULT_TEMPLATE = """You are an assistant tasked with taking a natural language \
query from a user and converting it into a query for a vectorstore. \
In this process, you strip out information that is not relevant for \
the retrieval task. Here is the user query: {question}"""
# 您的任务是将用户输入的自然语言查询转换为向向量存储库的查询。在这个过程中，你需要筛选掉与检索任务无关的信息。用户查询为：{question}

In [6]:
retriever_from_llm = RePhraseQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=qw_llm_openai
)

In [8]:
docs = retriever_from_llm.invoke(
    "Hi I'm Lance. What are the approaches to Task Decomposition?"
)

NotFoundError: Error code: 404 - {'error': {'code': 'model_not_supported', 'param': None, 'message': 'Unsupported model `qwen2-plus` for OpenAI compatibility mode.', 'type': 'invalid_request_error'}}

In [9]:
docs = retriever_from_llm.invoke(
    "I live in San Francisco. What are the Types of Memory?"
)

NotFoundError: Error code: 404 - {'error': {'code': 'model_not_supported', 'param': None, 'message': 'Unsupported model `qwen2-plus` for OpenAI compatibility mode.', 'type': 'invalid_request_error'}}

Custom prompt 定义提示

In [14]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with taking a natural languge query from a user
    and converting it into a query for a vectorstore. In the process, strip out all 
    information that is not relevant for the retrieval task and return a new, simplified
    question for vectorstore retrieval. The new user query should be in pirate speech.
    Here is the user query: {question} """,
)
llm_chain = QUERY_PROMPT | qw_llm_openai

In [15]:
retriever_from_llm_chain = RePhraseQueryRetriever(
    retriever=vectorstore.as_retriever(), llm_chain=llm_chain
)

ValidationError: 5 validation errors for RePhraseQueryRetriever
llm_chain -> prompt
  field required (type=value_error.missing)
llm_chain -> llm
  field required (type=value_error.missing)
llm_chain -> first
  extra fields not permitted (type=value_error.extra)
llm_chain -> last
  extra fields not permitted (type=value_error.extra)
llm_chain -> middle
  extra fields not permitted (type=value_error.extra)

In [ ]:
docs = retriever_from_llm_chain.invoke(
    "Hi I'm Lance. What is Maximum Inner Product Search?"
)